In [6]:
from helpers import chunk_and_contextualize_text 
import chromadb 
import ollama
chroma_client = chromadb.PersistentClient(path="chromadb/")
# chroma_client.delete_collection("my_collection")
collection = chroma_client.create_collection(name="my_collection", get_or_create=True)

In [7]:
count = 0
def add_to_chroma_db(text: str):
    global count 
    embed = ollama.embeddings(model='nomic-embed-text', prompt=text)
    collection.add(embeddings=[embed["embedding"]], documents=[text], ids=[f"id_{count}"])
    count += 1
def query_chroma_db(text: str, n_results=10):
    return collection.query(ollama.embeddings(model='nomic-embed-text', prompt=text)["embedding"], n_results=n_results)

In [8]:
%%time

# find all file in data directory 
import os
# iterate over all files in data directory
# add them into chromadb
for filename in os.listdir("data"):
    with open(f"data/{filename}", "r") as f:
        print(f"Processing {filename}")
        text = f.read()
        chunked_text = chunk_and_contextualize_text(text)
        for chunk in chunked_text:
            add_to_chroma_db(chunk)

Processing Allay.md


Processing chunks: 100%|██████████| 5/5 [00:03<00:00,  1.52it/s]


Processing Armadillo.md


Processing chunks: 100%|██████████| 9/9 [00:03<00:00,  2.61it/s]


Processing Axolotl.md


...

Processing Zombie_Villager.md


Processing chunks: 100%|██████████| 14/14 [00:08<00:00,  1.59it/s]


Processing Zombified_Piglin.md


Processing chunks: 100%|██████████| 12/12 [00:05<00:00,  2.12it/s]


CPU times: total: 1.3 s
Wall time: 9min 32s


In [9]:
def ask_rag(question, model='8k-llama3.2'):
    responses = query_chroma_db(question, n_results=10)
    # for response in responses['documents'][0]:
    #     print(response)
    context = ""
    for i in range(len(responses['documents'][0])):
        context += f"{i+1}. {responses['documents'][0][i]} "
    
    system = f"""
    You are a Minecraft expert. You are answering a question about the game of Minecraft.
    Use the following pieces of context to answer the question at the end. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer.
    Use five sentences maximum and keep the answer as concise as possible. 
    {context}
    """
    
    print(len(system))
        
    response = ollama.chat(model=model, messages=[
    {
        'role': 'system',
        'content': system,
    },
    {
        'role': 'user',
        'content': question,
    },
    ])
    
    return response['message']['content']

In [10]:
print(ask_rag("What is piglin brutes?", model='24k-llama3.2'))

13560
A piglin brute is a hostile, stronger variant of piglins that wields a golden axe and appears in all types of bastion remnants. They are distinct from regular piglins in several ways, including their behavior and attack patterns.
